<a href="https://colab.research.google.com/github/pallavkumar1981/MSDS462-self-driving-car/blob/main/03_Test_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructions

* Run the simulator on local jupyter notebook.
* Key considerations:
    - Force install openCV version '4.5.4'. Avoid version 4.6 and above
    - Download/unzip the image data (images and data.txt); and copy them under 'driving_dataset' directory
    - Dataset - https://drive.google.com/file/d/1PZWa6H0i1PCH9zuYcIh5Ouk_p-9Gh58B/view
    - Download the best performing model ('self_drive_model_E2E_v1.hdf5') under 'models' folder.
    - Make sure the steering wheel image is copied in the working directory
* Press 'q' to end simulation at any point



# 0. Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Normalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import tensorflow.keras.backend as K

import os
import random
from scipy import pi
from datetime import datetime
import cv2
import pickle
import json
from itertools import islice

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
cv2.__version__

'4.5.4'

# 1. Loading prepared data (train and validation) for model development

In [ ]:
def loadPreparedDataFromPickle():
    with open('./pickle/X_train.pkl', 'rb') as f:
        Xtrain = np.array(pickle.load(f))
    with open('./pickle/y_train.pkl', 'rb') as f:
        ytrain = np.array(pickle.load(f))
    with open('./pickle/X_val.pkl', 'rb') as f:
        Xval = np.array(pickle.load(f))
    with open('./pickle/y_val.pkl', 'rb') as f:
        yval = np.array(pickle.load(f))
    return Xtrain, ytrain, Xval, yval

In [ ]:
%%time
X_train, y_train, X_val, y_val = loadPreparedDataFromPickle()

Wall time: 35 s


# 2. Load the (best) model

In [ ]:
checkpoint_file_name = './models/self_drive_model_E2E_v1.hdf5'

In [ ]:
def custom_activation(x):
    return tf.multiply(tf.math.atan(x), 2) #  arctan range is [-pi/2, pi/2]. so we have to get the angle between [-pi, pi] so we are multiplying by 2

In [ ]:
%%time
#https://stackoverflow.com/questions/55779286/valueerror-unknown-activation-function-my-custom-activation-function
model_e2e = tf.keras.models.load_model(checkpoint_file_name, 
                                          custom_objects={'custom_activation':custom_activation})

Wall time: 6.2 s


## 3. Test Simulation - E2E learning model

Reference: https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_geometric_transformations/py_geometric_transformations.html

In [ ]:
#to rotate the steering image 
steering_img = cv2.imread('steering_wheel_image.jpg', 0)
rows, cols = steering_img.shape
smoothed_angle = 0

In [ ]:
i = len(X_train)
length = i

while(cv2.waitKey(10) != ord('q')):
    full_image = cv2.imread("./driving_dataset/" + str(i) + ".jpg")
    resized_frame = cv2.resize(full_image, (200,66))
    resized_frame = resized_frame/255.0
    resized_frame = np.expand_dims(resized_frame, axis=0)
    pred_degrees = (model_e2e.predict(resized_frame) * 180)/pi  #to convert the radians to degrees
    actual_degrees = y_val[i-length]*180/pi

    print("Predicted steering angle (degrees): " + str(pred_degrees) + " | Actual steering angle (degrees) = " + str(actual_degrees))
    cv2.imshow("frame", full_image)
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    smoothed_angle += 0.2 * pow(abs((pred_degrees - smoothed_angle)), 2.0 / 3.0) * (pred_degrees - smoothed_angle) / abs(pred_degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),float(-smoothed_angle),1)
    dst = cv2.warpAffine(steering_img,M,(cols,rows))
    cv2.imshow("steering wheel", dst)
    i += 1

cv2.destroyAllWindows()

Predicted steering angle (degrees): [[8.465838]] | Actual steering angle (degrees) = 8.67
Predicted steering angle (degrees): [[8.467807]] | Actual steering angle (degrees) = 20.869999999999997
Predicted steering angle (degrees): [[13.81431]] | Actual steering angle (degrees) = 12.5
Predicted steering angle (degrees): [[8.803147]] | Actual steering angle (degrees) = -7.460000000000001
Predicted steering angle (degrees): [[2.8803794]] | Actual steering angle (degrees) = 0.0
Predicted steering angle (degrees): [[6.2032065]] | Actual steering angle (degrees) = 9.18
Predicted steering angle (degrees): [[1.1883109]] | Actual steering angle (degrees) = -2.12
Predicted steering angle (degrees): [[-1.2299345]] | Actual steering angle (degrees) = -9.28
Predicted steering angle (degrees): [[2.3184676]] | Actual steering angle (degrees) = 4.24
Predicted steering angle (degrees): [[-5.225957]] | Actual steering angle (degrees) = -38.52
Predicted steering angle (degrees): [[-3.4602132]] | Actual st

Predicted steering angle (degrees): [[10.829831]] | Actual steering angle (degrees) = 0.0
Predicted steering angle (degrees): [[11.251577]] | Actual steering angle (degrees) = 25.71
Predicted steering angle (degrees): [[10.870388]] | Actual steering angle (degrees) = 19.46
Predicted steering angle (degrees): [[10.08034]] | Actual steering angle (degrees) = 3.53
Predicted steering angle (degrees): [[8.252663]] | Actual steering angle (degrees) = 1.51
Predicted steering angle (degrees): [[5.7936254]] | Actual steering angle (degrees) = 29.450000000000003
Predicted steering angle (degrees): [[3.9013517]] | Actual steering angle (degrees) = 53.040000000000006
Predicted steering angle (degrees): [[3.9884856]] | Actual steering angle (degrees) = -4.64
Predicted steering angle (degrees): [[3.9884856]] | Actual steering angle (degrees) = 15.030000000000001
Predicted steering angle (degrees): [[3.9884856]] | Actual steering angle (degrees) = 6.150000000000001
Predicted steering angle (degrees):